In [3]:
%pip install numpy
%pip install tensorflow

In [4]:

%pip install pandas

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\research\appdata\local\programs\python\python311\lib\site-packages (2.2.0)



In [5]:
%pip install pillow

Note: you may need to restart the kernel to use updated packages.


### ResNet

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

num_classes = 4         # for 2x2 jigsaw puzzle there are 4 possible positions for each piece

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [7]:
# Dataset Source Credit: https://www.kaggle.com/datasets/shivajbd/jigsawpuzzle

# Load the Jigsaw Puzzle dataset and split it into train, test and valid sets
import os
import pandas as pd
import numpy as np
import PIL
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define function to load dataset from csv files and images
def load_dataset(root_dir, csv_filename):
    data = pd.read_csv(csv_filename)
    images = []
    labels = []
    for _, row in data.iterrows():
        image_path = os.path.join(root_dir, row['image'])
        image = load_img(image_path, target_size=(200, 200))
        image = img_to_array(image) / 255.0
        images.append(image)

        # Convert string label to list of integers
        labels.append([int(x) for x in row['label'].split()])
    return images, labels

In [8]:
# Load data for 2x2 puzzle
train_images_2x2, train_labels_2x2 = load_dataset('../data/puzzle_2x2/train', '../data/puzzle_2x2/train.csv')
print("Train images 2x2 loaded: ", len(train_images_2x2))

In [ ]:
test_images_2x2, test_labels_2x2 = load_dataset('../data/puzzle_2x2/test', '../data/puzzle_2x2/test.csv')
print("Test images 2x2 loaded: ", len(test_images_2x2))

In [ ]:
valid_images_2x2, valid_labels_2x2 = load_dataset('../data/puzzle_2x2/valid', '../data/puzzle_2x2/valid.csv')
print("Valid images 2x2 loaded: ", len(valid_images_2x2))

In [ ]:
# Load data for 3x3 puzzle
train_images_3x3, train_labels_3x3 = load_dataset('../data/puzzle_3x3/train', '../data/puzzle_3x3/train.csv')
print("Train images 3x3 loaded: ", len(train_images_3x3))

In [ ]:
test_images_3x3, test_labels_3x3 = load_dataset('../data/puzzle_3x3/test', '../data/puzzle_3x3/test.csv')
print("Test images 3x3 loaded: ", len(test_images_3x3))

In [ ]:
valid_images_3x3, valid_labels_3x3 = load_dataset('../data/puzzle_3x3/valid', '../data/puzzle_3x3/valid.csv')
print("Valid images 3x3 loaded: ", len(valid_images_3x3))

In [ ]:
# Concatenate 2x2 and 3x3 datasets
train_images = train_images_2x2 + train_images_3x3
test_images = test_images_2x2 + test_images_3x3
valid_images = valid_images_2x2 + valid_images_3x3

In [ ]:
# Concatenate 2x2 and 3x3 datasets
train_labels = train_labels_2x2 + train_labels_3x3
test_labels = test_labels_2x2 + test_labels_3x3
valid_labels = valid_labels_2x2 + valid_labels_3x3

In [ ]:
# Print shapes of the datasets
print("Train data shape:", train_images.shape)
print("Test data shape:", test_images.shape)
print("Validation data shape:", valid_images.shape)

# Convert lists to numpy arrays
train_data = np.array(train_images)
test_data = np.array(test_images)
valid_data = np.array(valid_images)

# Print shapes of the datasets
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)
print("Validation data shape:", valid_data.shape)

In [ ]:
# Print shapes of the datasets
print("Train labels shape:", train_labels.shape)
print("Test labels shape:", test_labels.shape)
print("Validation labels shape:", valid_labels.shape)

# Convert lists to numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
valid_labels = np.array(valid_labels)

# Print shapes of the datasets
print("Train labels shape:", train_labels.shape)
print("Test labels shape:", test_labels.shape)
print("Validation labels shape:", valid_labels.shape)

In [ ]:
# Normalize pixel values to [0, 1]
train_data = train_data / 255.0
test_data = test_data / 255.0
valid_data = valid_data / 255.0

# Train the model
history = model.fit(train_data, train_labels, epochs=10, validation_data=(valid_data, valid_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels)
print("Test accuracy:", test_acc)

# Make predictions
predictions = model.predict(test_data)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have true labels (ground truth) in test_labels and predictions from your model in predictions

# Accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

# Precision
precision = precision_score(test_labels, predictions)
print("Precision:", precision)

# Recall
recall = recall_score(test_labels, predictions)
print("Recall:", recall)

# F1 Score
f1 = f1_score(test_labels, predictions)
print("F1 Score:", f1)